In [9]:
import sys
sys.path.append('../scripts/')
from ideal_robot import *
from scipy.stats import expon, norm,uniform

In [10]:
class Robot(IdealRobot):
    def __init__(self,pose,agent=None, sensor = None, color = "black",
                 noise_per_meter = 5, noise_std=math.pi/60, bias_rate_stds=(0.1,0.1),
                expected_stuck_time = 1e100,expected_escape_time = 1e-100,
                expected_kidnap_time = 1e100, kidnap_range_x = (-5.0,5.0),
                kidnap_range_y =(-5.0,5.0)):
        super().__init__(pose,agent,sensor,color)
        self.noise_pdf = expon(scale = 1.0/(1e-100 + noise_per_meter))
        self.distance_until_noise = self.noise_pdf.rvs()
        self.theta_noise = norm(scale = noise_std)
        self.bias_rate_nu = norm.rvs(loc=1.0, scale = bias_rate_stds[0])
        self.bias_rate_omega = norm.rvs(loc=1.0, scale = bias_rate_stds[1])
        
        self.stuck_pdf = expon(scale=expected_stuck_time)
        self.escape_pdf = expon(scale=expected_escape_time)
        self.is_stuck = False
        self.time_until_stuck = self.stuck_pdf.rvs()
        self.time_until_escape = self.escape_pdf.rvs()
        self.sensor_time = 2.0
        self.is_first = True
        self.kidnap_pdf = expon(scale = expected_kidnap_time)
        self.time_until_kidnap = self.kidnap_pdf.rvs()
        rx,ry = kidnap_range_x, kidnap_range_y
        self.kidnap_dist = uniform(loc=(rx[0],ry[0],0.0), scale = (rx[1]-rx[0],ry[1]-ry[0], 2*math.pi))
        self.const_nu = 0
        self.const_omega = 0
        
    def noise(self,pose,nu,omega,time_interval):
        self.distance_until_noise  -= abs(nu)*time_interval + self.r*abs(omega)*time_interval
        if self.distance_until_noise <= 0.0:
            self.distance_until_noise += self.noise_pdf.rvs()
            pose[2] += self.theta_noise.rvs()
            
        return pose
    
    def bias(self,nu,omega):
        return nu*self.bias_rate_nu, omega*self.bias_rate_omega
    
    def stuck(self,nu,omega,time_interval):
        if self.is_stuck:
            self.time_until_escape -= time_interval
            if self.time_until_escape <= 0.0:
                self.time_until_escape += self.escape_pdf.rvs()
                self.is_stuck = False
        else:
            self.time_until_stuck -=time_interval
            if self.time_until_stuck <= 0.0:
                self.time_until_stuck += self.stuck_pdf.rvs()
                self.is_stuck = True
                
        return nu * (not self.is_stuck), omega * (not self.is_stuck)
    
    def kidnap(self,pose, time_interval):
        self.time_until_kidnap -= time_interval
        if self.time_until_kidnap <= 0.0:
            self.time_until_kidnap += self.kidnap_pdf.rvs()
            return np.array(self.kidnap_dist.rvs()).T
        else:
            return pose
    def check(self):
        obs = self.sensor.data(self.pose) if self.sensor else None
        nu, omega = self.agent.decision(obs)
        self.const_nu = nu
        self.const_omega = omega
        #print(obs)
        #print([self.const_nu, self.const_omega])    
        
    def one_step(self,time_interval):
        if not self.agent: return

        if (self.sensor_time >= 0.1) & (self.is_first == False):
            self.sensor_time -= time_interval
        elif self.is_first == True:
            self.check()
            self.is_first = False
        else:
            self.check()
            self.sensor_time = 2.0
      
        nu = self.const_nu
        omega = self.const_omega
        nu, omega = self.bias(nu,omega)
        nu, omega = self.stuck(nu,omega,time_interval)
        self.pose = self.state_transition(nu,omega,time_interval, self.pose)
        self.pose = self.noise(self.pose, nu,omega, time_interval)
        self.pose = self.kidnap(self.pose, time_interval)

In [11]:
class Camera(IdealCamera):
    def __init__(self, env_map,distance_range = (0.5,6.0),direction_range=(-math.pi/3, math.pi/3),
                distance_noise_rate = 0.1, direction_noise=math.pi/90):
        super().__init__(env_map, distance_range, direction_range)
        
        self.distance_noise_rate = distance_noise_rate
        self.direction_noise = direction_noise
        
    def noise(self, relpos):
        ell = norm.rvs(loc = relpos[0], scale=relpos[0]*self.distance_noise_rate)
        phi = norm.rvs(loc = relpos[1], scale=self.direction_noise)
        return np.array([ell,phi]).T
    
    def data(self, cam_pose):
        observed = []
        for lm in self.map.landmarks:
            z = self.observation_function(cam_pose, lm.pos)
            if self.visible(z):
                z = self.noise(z)
                observed.append((z,lm.id))
                
        self.lastdata = observed
        print(observed)
        return observed

In [12]:
world = World(30,0.1, debug=True)

m = Map()
m.append_landmark(Landmark(-4,2))
m.append_landmark(Landmark(2,-3))
m.append_landmark(Landmark(3,3))
world.append(m)

circling = Agent(0.2,10.0/180*math.pi)

r = Robot(np.array([0,0,0]).T,sensor = Camera(m), agent=circling)
world.append(r)

world.draw()

[(array([ 3.28711722, -0.96009824]), 1), (array([3.94743714, 0.80083131]), 2)]
[(array([4.57886661, 0.60431985]), 2)]
[(array([3.33880625, 0.28371886]), 2)]
[(array([3.1292122 , 0.04082514]), 2)]
[(array([ 3.1302966 , -0.27211644]), 2)]
[(array([ 2.53640104, -0.67861631]), 2)]
[]
[(array([4.89469522, 0.98921462]), 0)]
[(array([5.80167599, 0.84107548]), 0)]
[(array([4.96464197, 0.5325367 ]), 0)]
[(array([4.08119637, 0.27334091]), 0)]
[(array([ 4.26100455, -0.07678778]), 0)]
[(array([ 3.57534477, -0.28397712]), 0)]
[(array([ 3.37235542, -0.79884662]), 0)]
[(array([6.15014729, 0.86271078]), 1)]


In [5]:
world = World(30,0.1, debug=False)

m = Map()
circling = Agent(0.2,10.0/180*math.pi)
straight = Agent(0.2, 0.0)    
r = Robot(np.array([0,0,0]).T,sensor = Camera(m), agent=circling)
world.append(r)


m.append_landmark(Landmark(np.array([2,-2,0]).T, agent = straight))
m.append_landmark(Landmark(np.array([-1,-3,0]).T, agent = circling))
m.append_landmark(Landmark(np.array([3,3,0]).T, agent = None))
world.append(m)          


world.draw()

TypeError: Landmark.__init__() got an unexpected keyword argument 'agent'